In [ ]:
import itertools
import numpy as np
import os
from tqdm import tqdm
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd


from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
import functools

%load_ext autoreload
%autoreload 2

from genpen.utils import Paper
import bezier
from dataclasses import asdict, dataclass, field

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
@dataclass
class CircleGrowerPruner(object):
    start_poly: Polygon
    rad_gen: None
    n_growth_candidates:int = 10
    
    def __post_init__(self):
        self.polys = [self.start_poly]
        
    @property
    def mp(self):
        return MultiPolygon(self.polys)
    
    @property
    def current_rad(self):
        return self.rad_gen()
    
    
    
    def get_touching_circle(
        self,
        max_attempts=1000,
    ):
        n_attempts = 0
        rad = self.rad_gen()
        while n_attempts < max_attempts:
            
            search_ring = self.mp.buffer(rad).boundary
            search_locs = np.arange(0, search_ring.length, 2*rad) 
            scrambled_search_locs = np.random.permutation(search_locs)
            for sl in scrambled_search_locs:
                pt = search_ring.interpolate(sl)
                c = pt.buffer(rad)
                if (not c.intersects(self.mp)):
                    return c
                n_attempts += 1
        
    def make_growth_candidates(self):
        self.growth_candidates = []
        for i in range(self.n_growth_candidates):
            cand = cgp.get_touching_circle()
            if cand is not None:
                self.growth_candidates.append(cand)
    
    def get_distance_from_center(self, geom):
        return geom.distance(self.mp.centroid)
    
    def get_distance_from_target(self, geom):
        return geom.distance(self.target)
    
    @property
    def candidate_distances(self):
        return [self.get_distance_from_target(gc) for gc in self.growth_candidates]
#         return [self.get_distance_from_center(gc) for gc in self.growth_candidates]
    
    def grow(self):
        self.make_growth_candidates()
        order = np.argsort(self.candidate_distances)
        best = self.growth_candidates[order[0]]
        self.polys.append(best)
        
    def prune_other(self, other_geom):
        distances = [p.distance(other_geom) for p in self.polys]
        order = np.argsort(distances)
        worst = order[0]
        self.polys.pop(worst)
        
    def prune(self,):
        distances = [p.hausdorff_distance(self.mp) for p in self.polys]
        order = np.argsort(distances)
        choice = order[0]
        self.polys.pop(choice)

In [ ]:
start_poly = Point(140,50).buffer(3)
rad_gen = ss.uniform(loc=1, scale=5).rvs
cgp = CircleGrowerPruner(start_poly=start_poly, rad_gen=rad_gen, n_growth_candidates=5)
cgp.target = Point(140, 280)


In [ ]:
for i in range(350):
    cgp.grow()

In [ ]:
for i in range(50):
    cgp.grow()

In [ ]:
cgp.mp

In [ ]:
bps = []
for p in cgp.mp:
    d = p.centroid.y
    buffer_dist = np.interp(d, [35, 320], [0, -2.5])
    bp = p.buffer(buffer_dist)
    bps.append(bp)

In [ ]:
bps = []
for p in cgp.mp:
    d = p.centroid.y
    clip_dist = np.interp(d, [35, 320], [0, gp.get_rad(p)*2])
    bp = p.difference(sa.translate(p, xoff=clip_dist))
    bps.append(bp)

In [ ]:
polys = gp.merge_Polygons(bps)

In [ ]:


fbps = []
for _p in polys:
    area = np.log10(_p.area)
    p = gp.Poly(_p)
    n_iters = int(np.interp(area, [-3, 4.3], [1, 230]))
    stp = gp.ScaleTransPrms(n_iters = n_iters, d_buffer=-0.4, d_translate_factor=0., angles=np.random.uniform(0,np.pi*2))
    db_mults = gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=6.5, step_mu=0.)
    db_mults[db_mults<0.] = 0.05
    stp.d_buffers *= db_mults
#     stp.angles = np.linspace(0, np.radians(360*0.7), len(stp.d_buffers))
    p.fill_scale_trans(**stp.prms)
    fbps.append(p)

fills = gp.merge_LineStrings([p.fill for p in fbps])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.stroke(1)
sk.geometry(polys.boundary)

sk.display(color_mode='none')

In [ ]:

sk.vpype('linemerge --tolerance 0.1 linemerge --tolerance 0.1 linesimplify --tolerance 0.1 linesort')
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/227_wax_wane.svg')